In [1]:
import os
import shutil
import random
import config
from PIL import Image

In [6]:
def resize_and_crop_image(input_path, output_path, size):
    with Image.open(input_path) as image:
        target_ratio = size[0] / size[1]
        image_ratio = image.width / image.height

        if image_ratio > target_ratio:
            new_width = int(target_ratio * image.height)
            left = (image.width - new_width) / 2
            top = 0
            right = left + new_width
            bottom = image.height
        else:
            new_height = int(image.width / target_ratio)
            left = 0
            top = (image.height - new_height) / 2
            right = image.width
            bottom = top + new_height

        image = image.crop((left, top, right, bottom))
        image = image.resize(size, ImageResampling.LANCZOS)

        image.save(output_path)

In [4]:
def create_subset_resize_and_split(source_dir, train_test_base_folder, category, size, n, split_ratio=0.2):
    image_files = [f for f in os.listdir(source_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg')) and not f.startswith('._')]
    random.shuffle(image_files)
    selected_files = image_files[:n]

    split_index = int(len(selected_files) * (1 - split_ratio))

    train_folder = os.path.join(train_test_base_folder, 'train', category)
    test_folder = os.path.join(train_test_base_folder, 'test', category)

    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)

    for i, filename in enumerate(selected_files):
        input_path = os.path.join(source_dir, filename)
        if i < split_index:
            output_path = os.path.join(train_folder, filename)
        else:
            output_path = os.path.join(test_folder, filename)

        try:
            resize_and_crop_image(input_path, output_path, size)
        except Exception as e:
            print(f"Error processing {input_path}: {e}")


In [5]:
size = (600, 600)
n_film = 9900
n_digital = 9900
create_subset_resize_and_split(config.tts_source_dir_film, config.tts_base_folder, 'film', size, n_film)
create_subset_resize_and_split(config.tts_source_dir_digital, config.tts_base_folder, 'digital', size, n_digital)

/var/folders/q7/5h2c80n91kv48jr01119s4t80000gn/T/ipykernel_34036/543045609.py:20: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image = image.resize(size, Image.ANTIALIAS)
